
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>




# Model Registry

MLflow Model Registry is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance.  This lesson explores how to manage models using the MLflow model registry.

This demo notebook will use scikit-learn on the Airbnb dataset, but in the lab you will use MLlib.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Learning Objectives:<br>

By the end of this lesson, you should be able to;

* Explain MLflow Model Registry components
* Register a model to MLflow Model Registry
* Manage model lifecycle programmatically
* Manage model lifecycle using MLflow UI
* Archive and delete models from MLflow Model Registry


 
<img src="https://files.training.databricks.com/images/icon_note_24.png"/> If you would like to set up a model serving endpoint, you will need <a href="https://docs.databricks.com/applications/mlflow/model-serving.html#requirements" target="_blank">cluster creation</a> permissions.

## 📌 Requirements

**Required Databricks Runtime Version:** 
* Please note that in order to run this notebook, you must use one of the following Databricks Runtime(s): **12.2.x-cpu-ml-scala2.12**

## Lesson Setup

The first thing we're going to do is to **run setup script**. This script will define the required configuration variables that are scoped to each user.

In [0]:
%run "./Includes/Classroom-Setup"

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02"

Validating the locally installed datasets:
| listing local files...(4 seconds)
| validation completed...(4 seconds total)

Creating & using the schema "charlie_ohara_4mi2_da_sml" in the catalog "hive_metastore"...(1 seconds)

Predefined tables in "charlie_ohara_4mi2_da_sml":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02

Setup completed (10 seconds)





## Model Registry

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow Experiment and Run produced the model), model versioning, stage transitions (e.g. from staging to production), annotations (e.g. with comments, tags), and deployment management (e.g. which production jobs have requested a specific model version).

MLflow Model Registry has the following features:

* **Central Repository:** Register MLflow models with the MLflow Model Registry. A registered model has a unique name, version, stage, and other metadata.
* **Model Versioning:** Automatically keep track of versions for registered models when updated.
* **Model Stage:** Assigned preset or custom stages to each model version, like “Staging” and “Production” to represent the lifecycle of a model.
* **Model Stage Transitions:** Record new registration events or changes as activities that automatically log users, changes, and additional metadata such as comments.
* **CI/CD Workflow Integration:** Record stage transitions, request, review and approve changes as part of CI/CD pipelines for better control and governance.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/model-registry.png" style="height: 400px; margin: 20px"/></div>

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> See <a href="https://mlflow.org/docs/latest/model-registry.html" target="_blank">the MLflow docs</a> for more details on the model registry.




## Registering a Model

The following workflow will work with either the UI or in pure Python.  This notebook will use pure Python.

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> Explore the UI throughout this lesson by clicking the "Models" tab on the left-hand side of the screen.




Train a model and log it to MLflow using <a href="https://mlflow.org/docs/latest/tracking.html#automatic-logging" target="_blank">autologging</a>. Autologging allows you to log metrics, parameters, and models without the need for explicit log statements.

There are a few ways to use autologging:

  1. Call **`mlflow.autolog()`** before your training code. This will enable autologging for each supported library you have installed as soon as you import it.

  2. Enable autologging at the workspace level from the admin console

  3. Use library-specific autolog calls for each library you use in your code. (e.g. **`mlflow.spark.autolog()`**)

Here we are only using numeric features for simplicity of building the random forest.

In [0]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import pandas as pd
# don't just have to use spark mL with ml flow, so we can demo using sklearn as an alternative
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

df = pd.read_csv("dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02/airbnb/sf-listings/airbnb-cleaned-mlflow.csv".replace("dbfs:/", "/dbfs/"))

# convert ineteger fields to double in case of missing values, which would cause error
int_cols = df.select_dtypes(include='int64').columns
df[int_cols] = df[int_cols].astype('float64')
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

# creates a new experiment with our first run
with mlflow.start_run(run_name="LR Model") as run:
    # stores all the different metrics for us without specifically defining rmse or r2
    mlflow.sklearn.autolog(log_input_examples=True, log_model_signatures=True, log_models=True)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    signature = infer_signature(X_train, lr.predict(X_train))

2024/02/09 16:33:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."





Create a unique model name so you don't clash with other workspace users. 

Note that a registered model name must be a non-empty UTF-8 string and cannot contain forward slashes(/), periods(.), or colons(:).

In [0]:
suffix = DA.unique_name("-")
model_name = f"sklearn-lr_{suffix}"
print(f"Model Name: {model_name}")

Model Name: sklearn-lr_charlie-ohara-4mi2-da-sml





Register the model.

In [0]:
run_id = run.info.run_id
model_uri = f"runs:/{run_id}/model"
# creates a model under the Machine Learning > Models tab
# explicitly register a version of model artifact here for reuse 
model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'sklearn-lr_charlie-ohara-4mi2-da-sml'.
2024/02/09 16:35:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sklearn-lr_charlie-ohara-4mi2-da-sml, version 1
Created version '1' of model 'sklearn-lr_charlie-ohara-4mi2-da-sml'.



## Model Registery in MLflow UI

**Open the *Models* tab on the left of the screen to explore the registered model.**  Note the following:<br><br>

* It logged who trained the model and what code was used
* It logged a history of actions taken on this model
* It logged this model as a first version

<div><img src="https://files.training.databricks.com/images/registered_model_v3.png" style="height: 600px; margin: 20px"/></div>




Check the status.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
model_version_details = client.get_model_version(name=model_name, version=1)

model_version_details.status

Out[9]: 'READY'




Now add a model description

In [0]:
client.update_registered_model(
    name=model_details.name,
    description="This model forecasts Airbnb housing list prices based on various listing inputs."
)

Out[10]: <RegisteredModel: creation_timestamp=1707496515899, description=('This model forecasts Airbnb housing list prices based on various listing '
 'inputs.'), last_updated_timestamp=1707496624011, latest_versions=[], name='sklearn-lr_charlie-ohara-4mi2-da-sml', tags={}>




Add a version-specific description.

In [0]:
client.update_model_version(
    name=model_details.name,
    version=model_details.version,
    # OLS = ordinary least squares
    description="This model version was built using OLS linear regression with sklearn."
)

Out[11]: <ModelVersion: creation_timestamp=1707496516259, current_stage='None', description='This model version was built using OLS linear regression with sklearn.', last_updated_timestamp=1707496638035, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='b6f076599d4b4ab3a2ad177c7e69c363', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/b6f076599d4b4ab3a2ad177c7e69c363/artifacts/model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='1'>




## Deploying a Model

The MLflow Model Registry defines several model stages: **`None`**, **`Staging`**, **`Production`**, and **`Archived`**. Each stage has a unique meaning. For example, **`Staging`** is meant for model testing, while **`Production`** is for models that have completed the testing or review processes and have been deployed to applications. 

Users with appropriate permissions can transition models between stages.




Now that you've learned about stage transitions, transition the model to the **`Production`** stage.

In [0]:
import time

time.sleep(10) # In case the registration is still pending

In [0]:
client.transition_model_version_stage(
    name=model_details.name,
    version=model_details.version,
    # model in production is not able to be deleted = protected 
    stage="Production"
)

Out[13]: <ModelVersion: creation_timestamp=1707496516259, current_stage='Production', description='This model version was built using OLS linear regression with sklearn.', last_updated_timestamp=1707496726914, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='b6f076599d4b4ab3a2ad177c7e69c363', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/b6f076599d4b4ab3a2ad177c7e69c363/artifacts/model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='1'>




Fetch the model's current status.

In [0]:
model_version_details = client.get_model_version(
    name=model_details.name,
    version=model_details.version
)
print(f"The current model stage is: '{model_version_details.current_stage}'")

The current model stage is: 'Production'





Fetch the latest model using a **`pyfunc`**.  Loading the model in this way allows us to use the model regardless of the package that was used to train it.

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> You can load a specific version of the model too.

In [0]:
import mlflow.pyfunc # used to load in all sort of models no matter if made by Spark ML or skikitlearn etc

model_version_uri = f"models:/{model_name}/1"

print(f"Loading registered model version from URI: '{model_version_uri}'")
model_version_1 = mlflow.pyfunc.load_model(model_version_uri)

Loading registered model version from URI: 'models:/sklearn-lr_charlie-ohara-4mi2-da-sml/1'





Apply the model.

In [0]:
model_version_1.predict(X_test) # equivalent with spark ml is transform

Out[16]: array([101.11471214, 157.97132179, 147.97169741, ..., 209.28368398,
       144.62247929, 131.32160683])




### Deploying a New Model Version

The MLflow Model Registry enables you to create multiple model versions corresponding to a single registered model. By performing stage transitions, you can seamlessly integrate new model versions into your staging or production environments.




Create a new model version and register that model when it's logged.

In [0]:
from sklearn.linear_model import Ridge

with mlflow.start_run(run_name="LR Ridge Model") as run:
    alpha = .9
    # use an alternative algorithm still using skikitlearn
    ridge_regression = Ridge(alpha=alpha)
    ridge_regression.fit(X_train, y_train)

    # Specify the `registered_model_name` parameter of the `mlflow.sklearn.log_model()`
    # function to register the model with the MLflow Model Registry. This automatically
    # creates a new model version

    mlflow.sklearn.log_model(
        sk_model=ridge_regression,
        artifact_path="sklearn-ridge-model",
        registered_model_name=model_name,
    )

    mlflow.log_params(ridge_regression.get_params())
    mlflow.log_metric("mse", mean_squared_error(y_test, ridge_regression.predict(X_test)))

Registered model 'sklearn-lr_charlie-ohara-4mi2-da-sml' already exists. Creating a new version of this model...
2024/02/09 16:41:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sklearn-lr_charlie-ohara-4mi2-da-sml, version 2
Created version '2' of model 'sklearn-lr_charlie-ohara-4mi2-da-sml'.





Put the new model into staging.

In [0]:
client.transition_model_version_stage(
    name=model_details.name,
    version=2,
    stage="Staging"
)

Out[18]: <ModelVersion: creation_timestamp=1707496872315, current_stage='Staging', description='', last_updated_timestamp=1707496903650, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='636a724f775b4d79a4da949b41b51dc3', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/636a724f775b4d79a4da949b41b51dc3/artifacts/sklearn-ridge-model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='2'>


### Review Model Using the UI


Check the UI to see the new model version.

<div><img src="https://files.training.databricks.com/images/model_version_v3.png" style="height: 400px; margin: 20px"/></div>




Use the search functionality to grab the latest model version.

In [0]:
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])
print(new_model_version)

2



### Update a Deployed Model

Add a description to this new version.

In [0]:
client.update_model_version(
    name=model_name,
    version=new_model_version,
    description=f"This model version is a ridge regression model with an alpha value of {alpha} that was trained in scikit-learn."
)

Out[21]: <ModelVersion: creation_timestamp=1707496872315, current_stage='Staging', description=('This model version is a ridge regression model with an alpha value of 0.9 '
 'that was trained in scikit-learn.'), last_updated_timestamp=1707496972776, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='636a724f775b4d79a4da949b41b51dc3', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/636a724f775b4d79a4da949b41b51dc3/artifacts/sklearn-ridge-model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='2'>




Since this model is now in staging, you can execute an automated CI/CD pipeline against it to test it before going into production.  Once that is completed, you can push that model into production.

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=new_model_version,
    stage="Production", 
    archive_existing_versions=True # Archive existing model in production to be replaced with new verion 2 
)

Out[22]: <ModelVersion: creation_timestamp=1707496872315, current_stage='Production', description=('This model version is a ridge regression model with an alpha value of 0.9 '
 'that was trained in scikit-learn.'), last_updated_timestamp=1707496997433, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='636a724f775b4d79a4da949b41b51dc3', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/636a724f775b4d79a4da949b41b51dc3/artifacts/sklearn-ridge-model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='2'>




Delete version 1.  

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> You cannot delete a model that is not first archived.

In [0]:
client.delete_model_version( # need to archive first (see step above)
    name=model_name,
    version=1
)




Archive version 2 of the model too.

In [0]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Archived"
)

Out[24]: <ModelVersion: creation_timestamp=1707496872315, current_stage='Archived', description=('This model version is a ridge regression model with an alpha value of 0.9 '
 'that was trained in scikit-learn.'), last_updated_timestamp=1707497048722, name='sklearn-lr_charlie-ohara-4mi2-da-sml', run_id='636a724f775b4d79a4da949b41b51dc3', run_link='', source='dbfs:/databricks/mlflow-tracking/3021276973847332/636a724f775b4d79a4da949b41b51dc3/artifacts/sklearn-ridge-model', status='READY', status_message='', tags={}, user_id='6043631322962989', version='2'>




Now delete the entire registered model.

In [0]:
client.delete_registered_model(model_name)




## Review
**Question:** How does MLflow tracking differ from the model registry?  
**Answer:** Tracking is meant for experimentation and development.  The model registry is designed to take a model from tracking and put it through staging and into production.  This is often the point that a data engineer or a machine learning engineer takes responsibility for the deployment process.

**Question:** Why do I need a model registry?  
**Answer:** Just as MLflow tracking provides end-to-end reproducibility for the machine learning training process, a model registry provides reproducibility and governance for the deployment process.  Since production systems are mission critical, components can be isolated with ACL's so only specific individuals can alter production models.  Version control and CI/CD workflow integration is also a critical dimension of deploying models into production.

**Question:** What can I do programmatically versus using the UI?  
**Answer:** Most operations can be done using the UI or in pure Python.  A model must be tracked using Python, but from that point on everything can be done either way.  For instance, a model logged using the MLflow tracking API can then be registered using the UI and can then be pushed into production.




## Additional Topics & Resources

**Q:** Where can I find out more information on MLflow Model Registry?  
**A:** Check out <a href="https://mlflow.org/docs/latest/model-registry.html" target="_blank">the MLflow documentation</a>


## Classroom Cleanup

Run the following cell to remove lessons-specific assets created during this lesson:

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the schema "charlie_ohara_4mi2_da_sml"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)


&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>